In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install groq

In [ ]:
from groq import Groq
!export GROQ_API_KEY = 'YOUR_GROQ_API_KEY'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
import re
import nltk
import os
#from langdetect import detect
#from googletrans import Translator
# Initialize Google Translator
#translator = Translator()

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## P1: i. Without Final Label on.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 16 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   Timestamp = A12 + A4                                                        7 non-null      object 
 1   How effectively did the employee communicate with you?                      7 non-null      float64
 2   How promptly did the employee respond to your inquiries or concerns?        7 non-null      float64
 3   How quickly did the employee response your messages ?                       7 non-null      float64
 4   How clear and understandable was the information provided by the employee?  7 non-null      float64
 5   Overall feedback on communication                                           7 non-null      object 
 6   How effectively did the employee understand your bus

In [ ]:
# Drop the Timestamp column
df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
df

,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,Overall feedback on communication,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,Overall feedback,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,Overall feedback.1,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Overall feedback.2,Final Feedback
0,5.0,1.0,4.0,3.0,this is test feedback,2.0,1.0,good,1.0,5.0,bad,2.0,5.0,good,quite good enough
1,5.0,4.0,3.0,2.0,good,4.0,4.0,bad,5.0,2.0,good,4.0,5.0,good,"good, but could be better"
2,4.0,4.0,5.0,5.0,"Yes, I am satisfied with the employee who hear...",3.0,5.0,I am pleased with the employee as everything w...,3.0,3.0,Good but there is room for improvement,1.0,4.0,Employee was frank and unprofessional but cour...,My problem has been smoothly solved. I love th...
3,3.0,1.0,2.0,4.0,"It was so-so, as the employee took her time an...",1.0,3.0,Employee did not understand my business requir...,1.0,3.0,"For business, ok. For technology, she is not g...",3.0,4.0,Employee was very professional and courteous,"I dont like this service, i still have problem."
4,3.0,3.0,3.0,3.0,"Overall good, could be better.",4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,2.0,My problem is not solved still,So-so
5,3.0,4.0,2.0,1.0,Employee was fast and prompt but could not sol...,4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,3.0,Good in only technology,Problem solved!
6,2.0,1.0,2.0,4.0,I understood the solution but the solution del...,1.0,1.0,Very very bad,1.0,1.0,Total fool,1.0,1.0,Worst service i ever had,I am leaving this service. I will be filing co...
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.dropna()
df.tail(3)

,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,Overall feedback on communication,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,Overall feedback,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,Overall feedback.1,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Overall feedback.2,Final Feedback
4,3.0,3.0,3.0,3.0,"Overall good, could be better.",4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,2.0,My problem is not solved still,So-so
5,3.0,4.0,2.0,1.0,Employee was fast and prompt but could not sol...,4.0,2.0,Employee understood the requirements but could...,4.0,2.0,She is knowledgeable about technology but she ...,2.0,3.0,Good in only technology,Problem solved!
6,2.0,1.0,2.0,4.0,I understood the solution but the solution del...,1.0,1.0,Very very bad,1.0,1.0,Total fool,1.0,1.0,Worst service i ever had,I am leaving this service. I will be filing co...


In [ ]:
# Loading and basic preprocessing of dataframe
#df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form.csv')
#df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
#df = df.dropna()

In [ ]:
# Combine all overall feedback columns into one text column
feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df.head(3)

<ipython-input-8-eb751b6d5840>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,Overall feedback on communication,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,Overall feedback,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,Overall feedback.1,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Overall feedback.2,Final Feedback,All_feedback
0,5.0,1.0,4.0,3.0,this is test feedback,2.0,1.0,good,1.0,5.0,bad,2.0,5.0,good,quite good enough,this is test feedback good bad good
1,5.0,4.0,3.0,2.0,good,4.0,4.0,bad,5.0,2.0,good,4.0,5.0,good,"good, but could be better",good bad good good
2,4.0,4.0,5.0,5.0,"Yes, I am satisfied with the employee who hear...",3.0,5.0,I am pleased with the employee as everything w...,3.0,3.0,Good but there is room for improvement,1.0,4.0,Employee was frank and unprofessional but cour...,My problem has been smoothly solved. I love th...,"Yes, I am satisfied with the employee who hear..."


In [ ]:
df.drop(feedback_columns, axis=1, inplace=True)
df.tail(3)

<ipython-input-9-b2bd19039fd7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(feedback_columns, axis=1, inplace=True)


,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Final Feedback,All_feedback
4,3.0,3.0,3.0,3.0,4.0,2.0,4.0,2.0,2.0,2.0,So-so,"Overall good, could be better. Employee unders..."
5,3.0,4.0,2.0,1.0,4.0,2.0,4.0,2.0,2.0,3.0,Problem solved!,Employee was fast and prompt but could not sol...
6,2.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,I am leaving this service. I will be filing co...,I understood the solution but the solution del...


In [ ]:
# Text preprocessing function
# Translate text if not in English
#def translate_text_if_needed(text):
#    try:
#        language = detect(text)
#        if language != 'en':
#            translated = translator.translate(text, dest='en')
#            return translated.text
#        return text
#    except Exception as e:
#        return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text

# Apply text preprocessing
#df['All_feedback'] = df['All_feedback'].apply(translate_text_if_needed)
df['All_feedback'] = df['All_feedback'].apply(preprocess_text)
df

<ipython-input-10-bb59e920c904>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['All_feedback'] = df['All_feedback'].apply(preprocess_text)


,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Final Feedback,All_feedback
0,5.0,1.0,4.0,3.0,2.0,1.0,1.0,5.0,2.0,5.0,quite good enough,test feedback good bad good
1,5.0,4.0,3.0,2.0,4.0,4.0,5.0,2.0,4.0,5.0,"good, but could be better",good bad good good
2,4.0,4.0,5.0,5.0,3.0,5.0,3.0,3.0,1.0,4.0,My problem has been smoothly solved. I love th...,yes satisfied employee heard cleared doubt pro...
3,3.0,1.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,4.0,"I dont like this service, i still have problem.",soso employee took time quite late answer prob...
4,3.0,3.0,3.0,3.0,4.0,2.0,4.0,2.0,2.0,2.0,So-so,overall good could better employee understood ...
5,3.0,4.0,2.0,1.0,4.0,2.0,4.0,2.0,2.0,3.0,Problem solved!,employee fast prompt could solve dpubts employ...
6,2.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,I am leaving this service. I will be filing co...,understood solution solution delivery employee...


In [ ]:
# Apply text preprocessing
#df['Final Feedback'] = df['Final Feedback'].apply(translate_text_if_needed)
df['Final Feedback'] = df['Final Feedback'].apply(preprocess_text)
df

<ipython-input-11-d58dc65bf09a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Final Feedback'] = df['Final Feedback'].apply(preprocess_text)


,How effectively did the employee communicate with you?,How promptly did the employee respond to your inquiries or concerns?,How quickly did the employee response your messages ?,How clear and understandable was the information provided by the employee?,How effectively did the employee understand your business requirements?,How effectively did the employee handle any issues or concerns you had?,How knowledgeable was the employee about the technology used?,How knowledgeable was the employee about the Business domain?,How professional was the employee during your interaction?,How courteous and respectful was the employee?,Final Feedback,All_feedback
0,5.0,1.0,4.0,3.0,2.0,1.0,1.0,5.0,2.0,5.0,quite good enough,test feedback good bad good
1,5.0,4.0,3.0,2.0,4.0,4.0,5.0,2.0,4.0,5.0,good could better,good bad good good
2,4.0,4.0,5.0,5.0,3.0,5.0,3.0,3.0,1.0,4.0,problem smoothly solved love service,yes satisfied employee heard cleared doubt pro...
3,3.0,1.0,2.0,4.0,1.0,3.0,1.0,3.0,3.0,4.0,dont like service still problem,soso employee took time quite late answer prob...
4,3.0,3.0,3.0,3.0,4.0,2.0,4.0,2.0,2.0,2.0,soso,overall good could better employee understood ...
5,3.0,4.0,2.0,1.0,4.0,2.0,4.0,2.0,2.0,3.0,problem solved,employee fast prompt could solve dpubts employ...
6,2.0,1.0,2.0,4.0,1.0,1.0,1.0,1.0,1.0,1.0,leaving service filing complaint,understood solution solution delivery employee...


In [ ]:
# Split data into features and target
X = df.drop('Final Feedback', axis=1)
y = df['Final Feedback']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
df['Final Feedback']

0                       quite good enough
1                       good could better
2    problem smoothly solved love service
3         dont like service still problem
4                                    soso
5                          problem solved
6        leaving service filing complaint
Name: Final Feedback, dtype: object

In [ ]:
# Define ColumnTransformer to separate numerical and text data processing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include='number').columns.tolist()),
        ('text', TfidfVectorizer(), 'All_feedback')
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
from sklearn.linear_model import LogisticRegression

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Classification Report:")
#print(classification_report(y_test, y_pred))

Accuracy: 0.0


In [ ]:
client = Groq(
    # This is the default and can be omitted
    api_key='YOUR_GROQ_API_KEY'
    #api_key=os.environ.get("GROQ_API_KEY"),
)

# Function to generate detailed feedback using OpenAI's GPT-4
def generate_detailed_feedback(numerical_data, textual_feedback):
    # Convert numerical_data to a string representation
    numerical_data_str = str(numerical_data)

    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data_str}\n\nTextual Feedback: {textual_feedback}\n\nPlease provide a very short final feedback according to the client, i.e, either positive or negative. Not both. If negative, then check if it is very negative. Give output in the form of either Positive, Mostly Positive, Negative or Mostly Negative. And then Improvements or Matters to focus upon:"
    chat_completion = client.chat.completions.create(
    messages = [
    {
        "role": "system",
        "content": "you are a helpful assistant."
    },
{
        "role": "user",
        "content": prompt, # Use the formatted prompt here
    },
    ],
    model="llama3-8b-8192" # Add the model keyword argument here
    )
    return chat_completion.choices[0].message.content.strip()  # Extract the content from the response

numerical_data = X.iloc[2].drop('All_feedback').to_dict()
textual_feedback = X.iloc[2]['All_feedback']
detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback)
print(detailed_feedback)

Based on the numerical ratings and textual feedback, I would categorize the overall feedback as:

Mostly Positive

The employee received high ratings for their communication, promptness, and clarity, with some areas for improvement in their understanding of business requirements, technology, and domain knowledge. The textual feedback also mentions that the employee was courteous and respectful.

Improvements or Matters to focus upon:

* Enhancing employee's understanding of business requirements, technology, and domain knowledge to better assist clients.
* Addressing issues of unprofessional behavior during interactions.


## P1: ii. With Final Label on.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form 1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 17 columns):
 #   Column                                                                      Non-Null Count  Dtype  
---  ------                                                                      --------------  -----  
 0   Timestamp = A12 + A4                                                        7 non-null      object 
 1   How effectively did the employee communicate with you?                      7 non-null      float64
 2   How promptly did the employee respond to your inquiries or concerns?        7 non-null      float64
 3   How quickly did the employee response your messages ?                       7 non-null      float64
 4   How clear and understandable was the information provided by the employee?  7 non-null      float64
 5   Overall feedback on communication                                           7 non-null      object 
 6   How effectively did the employee understand your bus

In [ ]:
# Drop the Timestamp column
df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
df

In [ ]:
df = df.dropna()
df.tail(3)

In [ ]:
# Loading and basic preprocessing of dataframe
#df = pd.read_csv('/content/drive/MyDrive/Sayan RP files/Datasets/Client Feedback Form.csv')
#df.drop('Timestamp = A12 + A4', axis=1, inplace=True)
#df = df.drop(df.columns[0], axis=1)
#df = df.dropna()

In [ ]:
# Combine all overall feedback columns into one text column
feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df.head(3)

In [ ]:
df['All_feedback'] = df['All_feedback'] +" "+ df['Final Feedback']
df

In [ ]:
df.drop(feedback_columns, axis=1, inplace=True)
df.tail(3)

In [ ]:
# Text preprocessing function
# Translate text if not in English
def translate_text_if_needed(text):
    try:
        language = detect(text)
        if language != 'en':
            translated = translator.translate(text, dest='en')
            return translated.text
        return text
    except Exception as e:
        return text

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text

# Apply text preprocessing
df['All_feedback'] = df['All_feedback'].apply(translate_text_if_needed)
df['All_feedback'] = df['All_feedback'].apply(preprocess_text)
df

In [ ]:
# Apply text preprocessing
df['Final Feedback'] = df['Final Feedback'].apply(translate_text_if_needed)
df['Final Feedback'] = df['Final Feedback'].apply(preprocess_text)
df

In [ ]:
# Split data into features and target
X = df.drop(['Final Feedback', 'Final Label'], axis=1)
y = df['Final Label']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
y

0    positive
1    positive
2    positive
3    negative
4    negative
5    positive
6    negative
Name: Final Label, dtype: object

In [ ]:
# Define ColumnTransformer to separate numerical and text data processing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include='number').columns.tolist()),
        ('text', TfidfVectorizer(), 'All_feedback')
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])
from sklearn.linear_model import LogisticRegression

# Fit the model
pipeline.fit(X_train, y_train)

# Predict on the test set
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
#print("Classification Report:")
#print(classification_report(y_test, y_pred))

In [ ]:
# Function to generate detailed feedback using OpenAI's GPT-4

from openai import OpenAI
def generate_detailed_feedback(numerical_data, textual_feedback):
    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data}\n\nTextual Feedback: {textual_feedback}\n\nPlease provide a detailed final feedback summary."

    response = openai.Completion.create(
        engine="gpt-3.5-turbo",
        prompt=prompt,
        max_tokens=150
    )

    return response.choices[0].text.strip()

# Example usage
# Assume OpenAI API Key is set
openai.api_key = 'YOUR_GROQ_API_KEY'

numerical_data = X.iloc[0].drop('All_feedback').to_dict()
textual_feedback = X.iloc[0]['All_feedback']
detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback)
print(detailed_feedback)

##P2 = Final Feedback only Positive or Negative

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC
import re
import nltk
from langdetect import detect
from googletrans import Translator
# Initialize Google Translator
translator = Translator()

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Load your feedback data into a pandas DataFrame
df = pd.read_csv('feedback_data.csv')

# Drop the Timestamp column
df.drop('Timestamp = A12 + A4  ', axis=1, inplace=True)

# Combine all overall feedback columns into one text column
feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df.drop(feedback_columns, axis=1, inplace=True)

# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text

# Apply text preprocessing
df['All_feedback'] = df['All_feedback'].apply(preprocess_text)

# Encode target variable
label_encoder = LabelEncoder()
df['Final feedback'] = label_encoder.fit_transform(df['Final feedback'])

# Split data into features and target
X = df.drop('Final feedback', axis=1)
y = df['Final feedback']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE for handling imbalanced data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define ColumnTransformer to separate numerical and text data processing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include=np.number).columns.tolist()),
        ('text', TfidfVectorizer(), 'All_feedback')
    ],
    remainder='drop'
)

# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train_res, y_train_res)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))


##P3 = Descriptive Final Feedback using GPT with label Positive or Negative

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
import re
import nltk
import openai

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


In [ ]:
# Load your feedback data into a pandas DataFrame
df = pd.read_csv('feedback_data.csv')
df

In [ ]:
# Drop the Timestamp column
df.drop('Timestamp', axis=1, inplace=True)

# Combine all overall feedback columns into one text column
feedback_columns = [col for col in df.columns if 'Overall feedback' in col]
df['All_feedback'] = df[feedback_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
df.drop(feedback_columns, axis=1, inplace=True)


In [ ]:
# Text preprocessing function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    filtered_text = ' '.join([lemmatizer.lemmatize(word) for word in tokens if word not in stop_words])
    return filtered_text

# Apply text preprocessing
df['All_feedback'] = df['All_feedback'].apply(preprocess_text)

In [ ]:
# Encode target variable
label_encoder = LabelEncoder()
df['Final feedback'] = label_encoder.fit_transform(df['Final feedback'])

# Split data into features and target
X = df.drop('Final feedback', axis=1)
y = df['Final feedback']

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# SMOTE for handling imbalanced data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Define ColumnTransformer to separate numerical and text data processing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), X.select_dtypes(include=np.number).columns.tolist()),
        ('text', TfidfVectorizer(), 'All_feedback')
    ],
    remainder='drop'
)

In [ ]:
# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20],
    'classifier__min_samples_split': [2, 5]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train_res, y_train_res)

# Best model from GridSearchCV
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Function to generate detailed feedback using OpenAI's GPT-4
def generate_detailed_feedback(numerical_data, textual_feedback):
    prompt = f"Using the following numerical ratings and feedback:\n\nNumerical Ratings: {numerical_data}\n\nTextual Feedback: {textual_feedback}\n\nPlease provide a detailed final feedback summary."

    response = openai.Completion.create(
        engine="gpt-4",
        prompt=prompt,
        max_tokens=150
    )

    return response.choices[0].text.strip()

# Example usage
numerical_data = X.iloc[0].drop('All_feedback').to_dict()
textual_feedback = X.iloc[0]['All_feedback']
detailed_feedback = generate_detailed_feedback(numerical_data, textual_feedback)
print(detailed_feedback)